In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install transformers[torch]
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [13]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, GenerationConfig
import evaluate
import pandas as pd
import numpy as np

cuda


In [5]:
from google.colab import auth

In [10]:
dataset_name = "/content/College_Classification.csv"
# To load a local CSV file, specify 'csv' as the path and the file path in data_files
dataset = load_dataset('csv', data_files=dataset_name, encoding='latin-1')

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
model_name = "google/flan-t5-base"
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [15]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(base_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [19]:
# Print the features (column names) of the dataset
print(dataset['train'].features)

{'College Code': Value('int64'), 'College Name': Value('string'), 'University': Value('string'), 'Location': Value('string'), 'Cutoff': Value('float64'), 'No of Courses': Value('int64'), 'Intake': Value('float64'), 'Fees': Value('float64'), 'Salary': Value('float64'), 'Rating': Value('float64'), 'Infrastructure': Value('float64'), 'Faculty': Value('float64'), 'Crowd': Value('float64'), 'Hostel': Value('string'), 'Government': Value('int64'), 'Autonomous': Value('int64'), 'Minority': Value('string'), 'Rank': Value('int64'), 'New/Ban': Value('int64'), 'Admission': Value('int64')}


In [30]:
i = 178
# Access the 'train' split as the dataset was loaded from a single file
# Assuming 'Location' is the text you want to summarize and 'Rank' is the human-evaluated value
location = dataset['train'][i]['Location']
rank = dataset['train'][i]['Rank']
college_name = dataset['train'][i]['College Name']
university = dataset['train'][i]['University']

# Ensure the model is on the correct device
base_model.to(device)

# Include more variables in the prompt
prompt = f"\nCollege Name: {college_name}\nUniversity: {university}\nLocation: {location}\nWhat is the rank of this college?"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
output = tokenizer.decode(base_model.generate(input_ids, max_new_tokens=200)[0], skip_special_tokens=True)

print(f"Input Prompt : {prompt}")
print("--------------------------------------------------------------------")
print("Human evaluated rank ---->")
print(rank)
print("---------------------------------------------------------------------")
print("Baseline model generated output : ---->")
print(output)

Input Prompt : 
College Name: Sardar Patel Memorial Society's Namdevrao Poreddiwar College of Engineering and Technology, Gadchiroli
University: Gondwana
Location: Gadchiroli
What is the rank of this college?
--------------------------------------------------------------------
Human evaluated rank ---->
301
---------------------------------------------------------------------
Baseline model generated output : ---->
tenth


In [32]:
def tokenize_function(examples):
    # Create the input prompts and target labels for the batch
    inputs = []
    labels = []
    for i in range(len(examples['College Name'])):
        # Construct the prompt for each example in the batch
        prompt = f"College Name: {examples['College Name'][i]}\nUniversity: {examples['University'][i]}\nLocation: {examples['Location'][i]}\nWhat is the rank of this college?"
        inputs.append(prompt)
        # Convert the Rank to a string for the label
        labels.append(str(examples['Rank'][i]))

    # Tokenize the batch of inputs
    # Use padding and truncation to handle varying input lengths
    tokenized_inputs = tokenizer(inputs, padding="max_length", truncation=True, return_tensors="pt")

    # Tokenize the batch of labels
    tokenized_labels = tokenizer(labels, padding="max_length", truncation=True, return_tensors="pt")

    # Return the tokenized inputs and labels
    return {'input_ids': tokenized_inputs['input_ids'], 'attention_mask': tokenized_inputs['attention_mask'], 'labels': tokenized_labels['input_ids']}

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove original columns that are no longer needed after tokenization
# Keep the columns that are part of the tokenized output if necessary, but typically remove original text columns
original_columns = dataset['train'].column_names
# Ensure 'attention_mask' is also kept as it's part of the tokenized output
columns_to_remove = [col for col in original_columns if col not in ['input_ids', 'labels', 'attention_mask']]

tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)

# You can keep this filtering if you only want a subset for faster experimentation
# tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Map:   0%|          | 0/387 [00:00<?, ? examples/s]

In [33]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_model_train = get_peft_model(base_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model_train))

trainable model parameters: 884736
all model parameters: 248462592
percentage of trainable model parameters: 0.36%


In [34]:
output_dir = "./peft-dialogue-summary-training"

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
)

In [35]:
peft_trainer = Trainer(
    model=peft_model_train,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

peft_trainer.train()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sharathpai107 (sharathpai107-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=325, training_loss=1.8288683143028845, metrics={'train_runtime': 468.2703, 'train_samples_per_second': 4.132, 'train_steps_per_second': 0.694, 'total_flos': 1330264630886400.0, 'train_loss': 1.8288683143028845, 'epoch': 5.0})

In [36]:
peft_model_path = "./peft-dialogue-summary-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [39]:
from peft import PeftModel

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
peft_model = PeftModel.from_pretrained(peft_model_base, peft_model_path, is_trainable=False)

# Ensure the PEFT model is on the correct device
peft_model.to(device)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Ensure the model is on the correct device
base_model.to(device)


print(f"Input Prompt : {prompt}")
print("--------------------------------------------------------------------")
print("Human evaluated rank ---->")
print(rank)
print("---------------------------------------------------------------------")
print("Baseline model generated output : ---->")
print(output)
print("Peft model generated summary : ---->")
print(peft_model_text_output)

Input Prompt : 
College Name: Sardar Patel Memorial Society's Namdevrao Poreddiwar College of Engineering and Technology, Gadchiroli
University: Gondwana
Location: Gadchiroli
What is the rank of this college?
--------------------------------------------------------------------
Human evaluated rank ---->
301
---------------------------------------------------------------------
Baseline model generated output : ---->
tenth
Peft model generated summary : ---->
301
